In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import evaluation
import full_model
import data_aug
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import h5py

In [5]:
#fileio.make_h5()
#fileio.augment_h5()

In [ ]:
import full_model
import model_tester

import importlib
importlib.reload(full_model)
importlib.reload(model_tester)


model_name = 'MO1'
seed = None


h5_file = 'data/working/data.h5'
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if seed:
    torch.manual_seed(seed)

if model_name == 'MO2':
    net = full_model.ReproductionNet()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
else:
    model_name = 'MO1'
    net = full_model.ConvBiLstm()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0001)

train_losses, test_losses = model_tester.train_model(h5_file, net, model_name, optimizer=optimizer,
                         device=device, seed=seed)